# AI-Assisted Doctor Recommendation and Appointment Booking System

## 🏥 Problem Statement: Fixing the Broken Appointment Booking Experience
Every year, **20M+ patients are booked with the wrong doctor** due to flawed scheduling processes. This leads to wasted time, incorrect care, and lost trust in the healthcare system.

Consider this:

**Average wait time on hold:** 35+ minutes

**88% of appointments** are still scheduled via phone

Patients often **leave or skip care entirely** when they can't get through

This is more than a scheduling issue — it’s a public health problem.

💡 **My vision:** No patient should ever be booked with the wrong provider again.

## 💡 The Solution: Intelligent AI-Powered Appointment Scheduling
In this notebook, I present a GenAI-powered approach to automate and optimize the scheduling workflow, using structured generation, few-shot prompting, and function calling.

The solution addresses three key areas:

Triage — Understand the patient's condition/symptoms to route to the right specialist

Availability — Match with the correct doctor based on specialty + availability

Booking — Schedule the appointment and send a confirmation via email

## ⚙️ GenAI Capabilities Demonstrated
This notebook showcases at least 3 GenAI capabilities:

✅ Structured output / JSON mode — to return machine-readable scheduling data

✅ Few-shot prompting — to improve accuracy of triage decisions

✅ Function calling — to interact with real-time scheduling APIs and send confirmation emails

✅ (Optional Bonus): If you include vector search or retrieval from a symptom-specialist mapping DB, that counts as RAG / vector search

## 🧪 How It Works: Step-by-Step Breakdown
Below, you'll find sections that walk through the entire workflow:

Input patient symptoms

Triage to correct department/specialist (via GenAI few-shot prompts)

Fetch availability and construct structured appointment slot (via JSON mode)

Function call to finalize booking + send email confirmation

Each section contains:

💬 Prompting logic

🧠 LLM reasoning

📤 Structured responses

📩 API interactions

## ✅ Outcome
By the end of this notebook, you'll see how we can:

Eliminate the 35-minute phone wait

Remove human error from triaging

Deliver instant confirmation to the right doctor

Reduce patient drop-off, delays, and medical misrouting

In [1]:
# !pip uninstall -qqy jupyterlab  # Remove unused conflicting packages
!pip install -U -q "google-genai==1.7.0"
!pip install secure-smtplib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 5.6 MB/s eta 0:00:00


Below Python code imports modules for sending emails:

smtplib: Provides functionality for sending emails using the Simple Mail Transfer Protocol (SMTP).
email.mime.text.MIMEText: Used to create email messages containing plain text.
email.mime.multipart.MIMEMultipart: Used to create email messages that can contain multiple parts (e.g., plain text and attachments).

In [2]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

In [3]:
from google import genai
from google.genai import types

genai.__version__

'1.7.0'

In [4]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

In [5]:
# Define a retry policy. The model might make multiple consecutive calls automatically
# for a complex query, this ensures the client retries if it hits quota limits.
from google.api_core import retry

is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

if not hasattr(genai.models.Models.generate_content, '__wrapped__'):
  genai.models.Models.generate_content = retry.Retry(
      predicate=is_retriable)(genai.models.Models.generate_content)

In [6]:
# ## Create Local Database
# Create a local SQLite database to store patient, doctor, appointment, and availability data.

%load_ext sql
%sql sqlite:///healthcare.db

Now we will create tables and insert values in it
The `doctors` table holds the core information about the medical professionals.
* The `availability` table indicates when specific doctors are available for booking.
* The `appointments` table records the details of the appointments that have been made, linking patients to specific doctors and dates.

These three tables together form the basic structure for managing doctor information, their availability, and scheduled appointments in your healthcare system database.

In [7]:
%%sql

CREATE TABLE IF NOT EXISTS doctors (
    doctor_id INTEGER PRIMARY KEY AUTOINCREMENT,
    first_name VARCHAR(255) NOT NULL,
    last_name VARCHAR(255) NOT NULL,
    specialty VARCHAR(255) NOT NULL,
    location VARCHAR(255) NOT NULL,
    address VARCHAR(255) NOT NULL
    -- You might include other relevant doctor information here,
    -- such as contact details, etc.
);

CREATE TABLE IF NOT EXISTS availability (
    availability_id INTEGER PRIMARY KEY AUTOINCREMENT,
    doctor_id INTEGER NOT NULL,
    available_date DATE NOT NULL,
    FOREIGN KEY (doctor_id) REFERENCES doctors (doctor_id)
);

CREATE TABLE IF NOT EXISTS appointments (
    patient_name VARCHAR(255) NOT NULL,
    doctor_id INTEGER NOT NULL,
    appointment_date DATE NOT NULL,
    FOREIGN KEY (doctor_id) REFERENCES doctors (doctor_id)
);

-- Insert data into the 'doctors' table
INSERT INTO doctors (first_name, last_name, specialty, location, address) VALUES
    ('Eleanor', 'Vance', 'wrist specialist', 'Boston', '789 Oak St, Boston, MA'),
    ('Michael', 'Chen', 'General Practitioner', 'Cambridge', '101 Pine Ave, Cambridge, MA'),
    ('Sophia', 'Rodriguez', 'Dermatologist', 'Boston', '456 Elm St, Boston, MA'),
    ('William', 'Davis', 'Cardiologist', 'Brookline', '222 Maple Dr, Brookline, MA'),
    ('Olivia', 'Brown', 'Pediatrician', 'Boston', '888 Willow Ln, Boston, MA'),
    ('James', 'Garcia', 'Neurologist', 'Cambridge', '333 Cedar Blvd, Cambridge, MA'),
    ('Derek', 'Shepherd', 'Wrist Specialist', 'Boston', '123 Main St, Boston, MA'),
    ('Rob', 'Sue', 'General Practitioner', 'Boston', '70 Commonwealth Ave, Boston, MA'),
    ('Greg', 'Wilson', 'Shoulder Specialist', 'Boston', '1600 Huntington Avenue, Boston, MA'),
    ('Alice', 'Smith', 'General Practitioner', 'Boston', '456 Oak Ave, Boston, MA'),
    ('Bob', 'Johnson', 'Dermatologist', 'Boston', '789 Pine Ln, Boston, MA'),
    ('Carol', 'Williams', 'Cardiologist', 'Boston', '101 Elm St, Boston, MA'),
    ('David', 'Brown', 'Pediatrician', 'Boston', '222 Maple Dr, Boston, MA'),
    ('Eve', 'Davis', 'Orthopedist', 'Boston', '333 Willow Ct, Boston, MA'),
    ('Frank', 'Miller', 'Neurologist', 'Boston', '444 Birch Rd, Boston, MA'),
    ('Grace', 'Wilson', 'Ophthalmologist', 'Boston', '555 Cedar Blvd, Boston, MA'),
    ('Henry', 'Moore', 'Urologist', 'Boston', '666 Spruce Way, Boston, MA'),
    ('Ivy', 'Taylor', 'Gastroenterologist', 'Boston', '777 Oak St, Boston, MA'),
    ('Jack', 'Thomas', 'Pulmonologist', 'Boston', '888 Pine Ave, Boston, MA');

-- Insert data into the 'availability' table for Eleanor Vance (Orthopedist)
INSERT INTO availability (doctor_id, available_date) VALUES
    ((select doctor_id from doctors WHERE first_name='Eleanor'), '2025-04-20');
INSERT INTO availability (doctor_id, available_date) VALUES
    ((select doctor_id from doctors WHERE first_name='Eleanor'), '2025-04-21');
INSERT INTO availability (doctor_id, available_date) VALUES
    ((select doctor_id from doctors WHERE first_name='Eleanor'), '2025-04-22');
INSERT INTO availability (doctor_id, available_date) VALUES
    ((select doctor_id from doctors WHERE first_name='Eleanor'), '2025-04-23');
INSERT INTO availability (doctor_id, available_date) VALUES
    ((select doctor_id from doctors WHERE first_name='Eleanor'), '2025-05-05');

-- Insert data into the 'availability' table for Michael Chen (General Practitioner)
INSERT INTO availability (doctor_id, available_date) VALUES
    ((select doctor_id from doctors WHERE first_name='Michael'), '2025-04-20');
INSERT INTO availability (doctor_id, available_date) VALUES
    ((select doctor_id from doctors WHERE first_name='Michael'), '2025-04-24');
INSERT INTO availability (doctor_id, available_date) VALUES
    ((select doctor_id from doctors WHERE first_name='Michael'), '2025-04-25');
INSERT INTO availability (doctor_id, available_date) VALUES
    ((select doctor_id from doctors WHERE first_name='Michael'), '2025-04-26');


 * sqlite:///healthcare.db
Done.
Done.
Done.
19 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

In [8]:
# ## Define Database Functions
# Define functions for database interactions, used by the AI agent via function calling.


import sqlite3

db_file = "healthcare.db"
db_conn = sqlite3.connect(db_file)

## Code Summary

**1. `map_injury_to_specialty(injury: str) -> List[str]`:**
   - Maps a patient's injury to a list of relevant medical specialties using a predefined dictionary and potentially embeddings for unknown injuries (simplified in the example).

**2. `search_doctors_by_specialty(specialty: str, location: str) -> List[Dict]`:**
   - Searches the `doctors` database table for doctors matching a given specialty and location.
   - Returns a list of dictionaries, each containing basic doctor information (ID, name, specialty, address).

**3. `check_doctor_availability(doctor_name: str, date: str) -> List[Dict]`:**
   - Checks the availability of a specific doctor (by name) on a given date.
   - Also lists other doctors in the same specialty who are available on that date.
   - Returns a dictionary containing the target doctor's availability and a list of other available doctors.

**4. `book_appointment(patient_name: str, doctor_name: str, appointment_date: str) -> Dict`:**
   - Books an appointment for a patient with a specified doctor on a given date.
   - Checks doctor availability before booking.
   - Adds a new record to the `appointments` database table.
   - Returns a dictionary indicating the success or failure of the booking.

**5. `send_email_confirmation(patient_name: str, doctor_name: str, appointment_date: str, recipient_email: str) -> str`:**
   - Sends an email confirmation to the patient after an appointment is booked.
   - Uses the `smtplib` library to connect to an SMTP server (Gmail in this case) and send the confirmation email.
   - Returns a message indicating whether the email was sent successfully.

## Note : Replace password or passkey in login method

In [9]:
from typing import List, Dict
# 1. Medical Specialty Mapping Function
def map_injury_to_specialty(injury: str) -> List[str]:
    """Map patient's injury to relevant medical specialties using both rules and embeddings"""
    medical_mapping = {
        "wrist pain": ["wrist specialist","orthopedics"],
        "migraine": ["neurology", "general practice"],
        "chest pain": ["cardiology", "emergency medicine"],
        "rash": ["dermatology"],
        "asthma": ["pulmonology", "allergy"],
        "depression": ["psychiatry", "psychology"]
    }
    
    # Fallback to embeddings for unknown injuries
    if injury.lower() not in medical_mapping:
        # Use embeddings to find similar known injuries
        embedding = genai.embed_content(
            model="models/embedding-001",
            content=injury,
            task_type="retrieval_document"
        ).embedding
        
        # This would normally query a vector store of medical knowledge
        # Simplified example:
        return ["general practice"]  # Default to GP for unknown cases
        
    return medical_mapping[injury.lower()]


def search_doctors_by_specialty(specialty: str, location: str) -> List[Dict]:
    """
    Search for doctors by medical specialty and location.
    Returns a list of doctors with their basic information (name, specialty, address).
    Note: Availability information is not available as the 'availability' table has been removed.
    """
    print(f' - DB CALL: search_doctors_by_specialty({specialty}, {location})')

    try:
        cursor = db_conn.cursor()
        cursor.execute("""
            SELECT doctor_id,
                   first_name || ' ' || last_name AS name,
                   specialty,
                   address
            FROM doctors
            WHERE specialty = ?
            AND location = ?
        """, (specialty, location))

        doctors = []
        for row in cursor.fetchall():
            doctor = {
                "doctor_id": row[0],
                "name": row[1],
                "specialty": row[2],
                "address": row[3],
            }
            doctors.append(doctor)

        return doctors

    except sqlite3.Error as e:
        print(f"Database error: {str(e)}")
        return []

def check_doctor_availability(doctor_name: str, date: str) -> List[Dict]:
    """
    Check availability for a specific doctor (by name) on a given date and also
    list other doctors in the same specialty who are available on that date.
    Returns a dictionary containing the target doctor's availability and a list
    of other available doctors in the same specialty.
    """
    print(f' - DB CALL: check_doctor_availability({doctor_name}, {date})')

    try:
        cursor = db_conn.cursor()

        # Get the doctor's ID, specialty, and location by name
        cursor.execute("""
            SELECT doctor_id, specialty, location
            FROM doctors
            WHERE first_name || ' ' || last_name = ?
        """, (doctor_name,))
        doctor_info = cursor.fetchone()

        if not doctor_info:
            return {"error": f"Doctor with name '{doctor_name}' not found"}

        doctor_id = doctor_info[0]
        specialty = doctor_info[1]
        location = doctor_info[2]

        # Get availability for the specific doctor
        cursor.execute("""
            SELECT available_date
            FROM availability
            WHERE doctor_id = ?
            AND available_date = ?
        """, (doctor_id, date))
        availability = [{"available_date": row[0]} for row in cursor.fetchall()]

        other_doctors = []
        # Find other doctors in the same specialty available on the same date
        cursor.execute("""
            SELECT d.doctor_id, d.first_name || ' ' || d.last_name AS name
            FROM doctors d
            JOIN availability a ON d.doctor_id = a.doctor_id
            WHERE d.specialty = ?
              AND d.location = ?
              AND a.available_date = ?
              AND d.first_name || ' ' || d.last_name != ?
        """, (specialty, location, date, doctor_name))

        other_doctors = [{"doctor_id": row[0], "name": row[1]} for row in cursor.fetchall()]

        return {
            "availability": availability if availability else [{"status": "No availability for this doctor"}],
            "other_available_doctors": other_doctors
        }

    except ValueError:
        print("Invalid date format. Use YYYY-MM-DD")
        return {"error": "Invalid date format"}
    except sqlite3.Error as e:
        print(f"Database error: {str(e)}")
        return {"error": str(e)}

def book_appointment(patient_name: str, doctor_name: str, appointment_date: str) -> Dict:
    """Books an appointment for a patient with a specific doctor on a given date."""
    print(f' - DB CALL: book_appointment({patient_name}, {doctor_name}, {appointment_date})')
    try:
        cursor = db_conn.cursor()

        # Get doctor's ID from the name
        cursor.execute("""
            SELECT doctor_id
            FROM doctors
            WHERE first_name || ' ' || last_name = ?
        """, (doctor_name,))
        doctor_info = cursor.fetchone()

        if not doctor_info:
            return {"error": f"Doctor with name '{doctor_name}' not found"}
        doctor_id = doctor_info[0]

        # Check if the doctor is available on the given date
        cursor.execute("""
            SELECT available_date
            FROM availability
            WHERE doctor_id = ?
            AND available_date = ?
        """, (doctor_id, appointment_date))
        availability = cursor.fetchone()

        if not availability:
            return {"error": f"Doctor '{doctor_name}' is not available on {appointment_date}"}

        # Insert the appointment into an 'appointments' table (assuming it exists)
        cursor.execute("""
            INSERT INTO appointments (patient_name, doctor_id, appointment_date)
            VALUES (?, ?, ?)
        """, (patient_name, doctor_id, appointment_date))
        db_conn.commit()

        return {
            "success": True,
            "message": f"Appointment booked for {patient_name} with Dr. {doctor_name} on {appointment_date}"
        }

    except sqlite3.Error as e:
        db_conn.rollback()
        print(f"Database error: {str(e)}")
        return {"error": str(e)}

def send_email_confirmation(patient_name: str, doctor_name: str, appointment_date: str, recipient_email: str) -> str:
    """Sends an email confirmation for the appointment."""
    print(f' - FUNCTION CALL: send_email_confirmation({patient_name}, {doctor_name}, {appointment_date}, {recipient_email})')

    subject = "Appointment Confirmation"
    body = f"""
Dear {patient_name},

Your appointment has been successfully booked:
- Doctor: Dr. {doctor_name}
- Date: {appointment_date}

Thank you for using our service.

Best regards,
Healthcare System
"""

    msg = MIMEMultipart()
    msg['From'] = 'hemantoptional@gmail.com'
    msg['To'] = recipient_email
    msg['Subject'] = subject
    msg.attach(MIMEText(body, 'plain'))

    try:
        with smtplib.SMTP_SSL('smtp.gmail.com', 465) as server:  # Use 'as server'
            # server.ehlo()  # Not always necessary with SMTP_SSL
            server.login('hemantoptional@gmail.com', 'chln luic svyq vjrr') #replace with passkey
            server.sendmail('hemantoptional@gmail.com', recipient_email, msg.as_string())  # Use the sender's email directly
        return f"Email confirmation sent to {recipient_email}"
    except Exception as e:
        return f"Error sending email: {e}"


In [10]:
# These are the Python functions defined above.
db_tools=[map_injury_to_specialty,search_doctors_by_specialty,check_doctor_availability,book_appointment,send_email_confirmation]
# 3. System Instructions
instruction = """You are a medical guidance assistant that helps patients find appropriate care. Follow these steps:

1. Interpret the patient's described symptoms/injury
2. Determine appropriate medical specialties using map_injury_to_specialty
3. Search for doctors in that specialty using search_doctors_by_specialty
4. Check if doctor is available using check_doctor_availability if no doctor available then recommend General Practitioner in that areac
5. Book an appointment with the doctor using book_appointment
6. Ask user for email address and send an email to user's email address using send_email_confirmation 
5. Present options with locations

For emergencies (chest pain, stroke symptoms, etc.):
- Immediately call get_emergency_instructions
- Recommend visiting ER

Always:
- Maintain empathetic tone
"""

# 4. Example Usage
client = genai.Client(api_key=GOOGLE_API_KEY)
# Start a chat with automatic function calling enabled.
chat = client.chats.create(
    model="gemini-2.0-flash",
    config=types.GenerateContentConfig(
        system_instruction=instruction,
        tools=db_tools,
    ),
)
history=[]


In [11]:
patient_input="I fell and have severe wrist pain in Boston"
response = chat.send_message(
    patient_input
)
history.append({"type": "condition", "answer": patient_input})
print("\nStep 1: Patient describes condition")
print(response.text)


Step 1: Patient describes condition
OK. Based on your wrist pain, I recommend seeing a wrist specialist or an orthopedist. I can search for doctors in either of those specialties in Boston. Would you prefer a wrist specialist or an orthopedist?



In [12]:
patient_input="Please check if wrist specialist is available on 2025-05-05 if yes then book and appointment"
response = chat.send_message(
    patient_input
)
history.append({"type": "condition", "answer": patient_input})
print("\nStep 1: Patient describes condition")
print(response.text)

 - DB CALL: search_doctors_by_specialty(wrist specialist, Boston)
 - DB CALL: check_doctor_availability(Eleanor Vance, 2025-05-05)

Step 1: Patient describes condition
Great! Eleanor Vance is available on 2025-05-05. Can I book an appointment for you? I will need your name.



In [13]:
patient_input=" My name is Hemant Gaikwad "
response = chat.send_message(
    patient_input
)
history.append({"type": "condition", "answer": patient_input})
print("\nStep 1: Patient describes condition")
print(response.text)

 - DB CALL: book_appointment(Hemant Gaikwad, Eleanor Vance, 2025-05-05)

Step 1: Patient describes condition
I have booked an appointment for you with Eleanor Vance on 2025-05-05. Finally, I will send you an email confirmation. What is your email address?



In [14]:
patient_input=" my email address is hemant.gaikwad@somaiya.edu "
response = chat.send_message(
    patient_input
)
history.append({"type": "condition", "answer": patient_input})
print("\nStep 1: Patient describes condition")
print(response.text)

 - FUNCTION CALL: send_email_confirmation(Hemant Gaikwad, Eleanor Vance, 2025-05-05, hemant.gaikwad@somaiya.edu)

Step 1: Patient describes condition
OK, Hemant. I have sent an email confirmation to hemant.gaikwad@somaiya.edu. Is there anything else I can help you with?

